In [1]:
import numpy as np
import glob
import cv2

XYZ = np.load("/root/share/temp/aruco/data/XYZ.npy")
RPY = np.load("/root/share/temp/aruco/data/RPY.npy")

frames = []
for file in sorted(glob.glob("/root/share/temp/aruco/data/shisa*png")):
    frame = cv2.imread(file, cv2.IMREAD_UNCHANGED)
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2RGBA)
    frames.append(frame)

In [2]:
print(len(XYZ), len(RPY), len(frames))

94 94 94


- 0番目から始まる8文字読み込んで、nを得る
- 8番目から始まるn文字を読んで、XYZRPYを得る
- 8+n番目から始まる残り全部を読んで、imgを得る

In [1]:
import socket

t = 0
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(('0.0.0.0', 5900))
    s.listen(1)
    while True:
        conn, addr = s.accept()
        print("ac", end=" ")
        with conn:
            while True:
                data = conn.recv(1024)
                if not data:
                    break
                img_str = cv2.imencode(".png", frames[t])[1].tostring()

                msg = ("{} " * 6)[:-1].format(*XYZ[t], *RPY[t]).encode()  # [:-1]
                msglen = len(msg)
                msg = "{:04}    ".format(msglen).encode() + msg
                conn.sendall(msg + img_str)

                t += 1
                t %= len(XYZ)